In [ ]:
import geopandas as gpd
import folium
from utils import ras_output_wse_shp_to_nc
import pandas as pd
import numpy as np

In [ ]:
wse_gdf = gpd.read_file(r'Z:\js\ResMap\py\output\2yr Existing\tempfiles\ras_wse.shp')
wse_gdf.to_crs(epsg=4326, inplace=True)
# Compute depth as [max_wse - min_elev]
wse_gdf['depth_max'] = wse_gdf['wse_max'] - wse_gdf['min_elev']
wse_gdf['depth_max'] = wse_gdf['depth_max'].round(2)
wse_gdf.crs

In [ ]:
max_wse_gdf = wse_gdf[['geometry', 'min_elev', 'wse_max', 'depth_max']]
max_wse_gdf.shape

In [ ]:
max_wse_gdf = max_wse_gdf[max_wse_gdf['depth_max'] > 0]
max_wse_gdf.shape

In [ ]:
f = folium.Figure(width=800, height=550)
m = folium.Map(location=[30.87, -93.29],tiles='cartodbpositron', zoom_start=13, min_zoom = 10, max_zoom=15).add_to(f)

In [ ]:
wse_style = lambda x: {
    "color": "#E5214E",
    "fill": False,
    "opacity": 1
}
clip_style = lambda x: {
    "color": "#21E5D6",
    "fill": False,
    "opacity": 1
}

In [ ]:
g = folium.GeoJson(data=max_wse_gdf, style_function=wse_style).add_to(m)
# folium.GeoJsonTooltip(
#     fields=["depth_max", "wse_max", "min_elev"],
#     aliases=["Depth:", "WSE:", "Elevation:"]
# ).add_to(g)

m

In [ ]:
# Clip to Ras Mapper extracted Depth Extent
bounds_gdf = gpd.read_file(r'Z:\js\ResMap\data\westPark\2yr Existing Depth.geojson')
bounds_gdf.crs

In [ ]:
tooltip_clip_gdf = gpd.clip(max_wse_gdf, bounds_gdf) 

In [ ]:
g = folium.GeoJson(data=tooltip_clip_gdf, style_function=clip_style).add_to(m)
folium.GeoJsonTooltip(
    fields=["depth_max", "wse_max", "min_elev"],
    aliases=["Depth:", "WSE:", "Elevation:"]
).add_to(g)

m

In [ ]:
import xarray as xr
nc_file = r'Z:\js\ResMap\py\output\2yr Existing\RAS_WSE_Timeseries.nc'
ds = xr.open_dataset(nc_file, decode_cf=False )
ds


In [ ]:
ds.close()

In [ ]:
ds['values'][0].plot()

In [ ]:
# 
# ds['values'][0]
import datetime
import pandas as pd
import os
import numpy as np

points_csv = pd.read_csv(r'Z:\js\ResMap\py\ts_points.txt')
times = ds['time'].values
for station in range(len(ds['values'])):
    # print (ds['values'][station].values)
    
    name = points_csv[points_csv['point_id'] == ds['point_id'][station].values].point_name.values[0]
    values = ds['values'][station].values
    
    # print (times)
    df = pd.DataFrame({'time': times, 'values': values})
    df['datetime'] = df['time'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    df.drop(columns=['time'], inplace=True)
    df['values'].replace({-9999.0:np.NaN}, inplace=True)
    # convert to json
    json_file = os.path.join(r'Z:\js\ResMap\data\westPark\timeseries', f'{name} timeseries.json')
    
    df.to_json(json_file, orient='records')
# df.plot(x = 'datetime', y = 'values')
df['values'].max

In [ ]:
df.plot()

In [ ]:
import pandas as pd
# convert to dataframe
df = ds.to_dataframe()
df.index
# df['datetime'] = pd.to_datetime(df["time"], unit='s')
# convert to json
# json_file = os.path.join(args.datadirectory, f'{args.forecastname}_timeseries.json')

In [ ]:
ds.close()

In [ ]:
hdf_filename = 'S:\\For_Myles\\WestPark\\HEC-RASV6.3.1\\WestPark.p11.hdf'
hdf_interval = '30MIN'
hdf_starttime = '01Jan3000 00:00:00'
poly_wse_shp = 'Z:\\js\\ResMap\\py\\output\\2yr Existing\\tempfiles\\ras_wse.shp'
stations_locations_txt = 'Z:\\js\\ResMap\\py\\ts_points.txt'
output_nc = 'Z:\\js\\ResMap\\py\\output\\2yr Existing\\RAS_WSE_Timeseries.nc'


# ras_output_wse_shp_to_nc.wse_shp_to_nc(poly_wse_shp, stations_locations_txt, hdf_filename, output_nc)

In [ ]:
from scipy.spatial import cKDTree
wse_gdf = gpd.read_file(poly_wse_shp)
crs = wse_gdf.crs
wse_gdf.to_crs(epsg=4326, inplace=True)

station_locations = pd.read_csv(stations_locations_txt)

hiway_gdf = gpd.GeoDataFrame(
    station_locations, geometry=gpd.points_from_xy(station_locations['x'], station_locations['y']))
hiway_gdf.crs = 'epsg:4326'





In [ ]:
# Create the geometry x & y from the centroid of each feature in the wse shp file.
wse_gdf["x"] = wse_gdf.centroid.x
wse_gdf["y"] = wse_gdf.centroid.y

# Create a point feature geoDataframe from the centroid geometry.
wse_points = wse_gdf.copy()
wse_points['geometry'] = wse_points['geometry'].centroid

In [ ]:
# Get a dataframe of the closest wse to each station location.
# nearest_gdf = ckdnearest(hiway_gdf, wse_points)

gdA = hiway_gdf
gdB = wse_points

nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
btree = cKDTree(nB)
dist, idx = btree.query(nA, k=1)
gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
gdf = pd.concat(
    [
        gdA.reset_index(drop=True),
        gdB_nearest,
        pd.Series(dist, name='dist')
    ], 
    axis=1)

In [ ]:
gdf

In [ ]:
f = folium.Figure(width=800, height=550)
m = folium.Map(location=[30.87, -93.29],tiles='cartodbpositron', zoom_start=13, min_zoom = 10, max_zoom=15).add_to(f)

In [ ]:
# g = folium.GeoJson(data=gdB).add_to(m)
g = folium.GeoJson(data=gdA).add_to(m)
m


In [ ]:
wse_points.to_file(r'Z:\js\ResMap\py\output\2yr Existing\tempfiles\wse_points.shp')

# gdf


In [ ]:
df2 = gdf.T.groupby(level=0).first().T

df2

In [ ]:
gdf2 = gpd.GeoDataFrame(df2, geometry="geometry")
gdf2.to_file(r'Z:\js\ResMap\py\output\2yr Existing\tempfiles\wse_points_nearest.shp')


In [ ]:
gdf_ts = gdf2.drop(columns=['Area2D', 'Cell_Index', 'Easting', 'Northing', 'dist', 'geometry',
       'min_elev', 'point_id', 'point_name', 'type', 'wse_max', 'x', 'y'])

In [ ]:
gdf_ts.loc[:].values

In [ ]:
hiway_gdf.to_file(r'Z:\js\ResMap\py\output\2yr Existing\tempfiles\hiway_gdf.shp')
hiway_gdf